In [1]:
# Load environment variables from the .env file
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.llms import Ollama
from langchain_openai import OpenAI

from langchain.tools import tool
from langchain import hub
from langchain.prompts import load_prompt

from langchain.agents import AgentExecutor, create_react_agent

from langchain.chains import APIChain
from langchain.chains.api import open_meteo_docs


In [14]:
# Switch to use openAI
#llm = OpenAI(temperature=0, verbose=True, model="gpt-3.5-turbo-instruct") # defautl model : "gpt-3.5-turbo-instruct"
llm = Ollama(model="mistral")

In [15]:
# custom simple tool
@tool
def square(x: int) -> int:
    """
    squares a number
    """
    return int(x)**2

In [5]:
# tool features
print(square.name)
print(square.description)
print(square.args)

square
square(x: int) -> int - squares a number
{'x': {'title': 'X', 'type': 'integer'}}


In [6]:
# pull prompt from the hub and saves it 
obj = hub.pull("hwchase17/react")
obj.save("prompt.json")

In [16]:
# inititalize prompt and tools for custom agent
prompt = load_prompt('prompt.json')
tools = [
    square
    ]

In [8]:
# the prompt template
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [17]:
agent_custom = create_react_agent(
     llm,
     tools,
    prompt
    )

agent_executor = AgentExecutor(agent=agent_custom, tools=tools, verbose=True)

In [18]:
agent_executor.invoke({"input":"What is the square of 5 "})



> Entering new AgentExecutor chain...
 I need to find the square of the number 5.
Action: square
Action Input: 525 The observation tells me that the square of 5 is 25.
Final Answer: The square of 5 is 25.

> Finished chain.


{'input': 'What is the square of 5 ', 'output': 'The square of 5 is 25.'}

In [22]:
open_meteo_docs.OPEN_METEO_DOCS

'BASE URL: https://api.open-meteo.com/\n\nAPI Documentation\nThe API endpoint /v1/forecast accepts a geographical coordinate, a list of weather variables and responds with a JSON hourly weather forecast for 7 days. Time always starts at 0:00 today and contains 168 hours. All URL parameters are listed below:\n\nParameter\tFormat\tRequired\tDefault\tDescription\nlatitude, longitude\tFloating point\tYes\t\tGeographical WGS84 coordinate of the location\nhourly\tString array\tNo\t\tA list of weather variables which should be returned. Values can be comma separated, or multiple &hourly= parameter in the URL can be used.\ndaily\tString array\tNo\t\tA list of daily weather variable aggregations which should be returned. Values can be comma separated, or multiple &daily= parameter in the URL can be used. If daily weather variables are specified, parameter timezone is required.\ncurrent_weather\tBool\tNo\tfalse\tInclude current weather conditions in the JSON output.\ntemperature_unit\tString\tNo

In [20]:
chain = APIChain.from_llm_and_api_docs(
    llm,
    open_meteo_docs.OPEN_METEO_DOCS,
    verbose=True,
    limit_to_domains=["https://api.open-meteo.com/"],
)

In [21]:
chain.invoke(
    "What is the weather like right now in Munich, Germany in degrees Fahrenheit?"
)



> Entering new APIChain chain...
 Based on the user question and the given API documentation, the following API URL can be generated to get the current temperature in Fahrenheit for Munich, Germany:

`https://api.open-meteo.com/v1/forecast?latitude=48.1352&longitude=11.5761&temperature_unit=fahrenheit&current_weather=true`

This API call requests the hourly weather forecast for Munich with a geographical coordinate (latitude and longitude) set to its actual value, the temperature unit set to Fahrenheit (fahrenheit), and the current weather data included in the JSON output (current\_weather=true). By including only these necessary parameters, we can get a response that is as short as possible while still answering the user question.


ValueError: Based on the user question and the given API documentation, the following API URL can be generated to get the current temperature in Fahrenheit for Munich, Germany:

`https://api.open-meteo.com/v1/forecast?latitude=48.1352&longitude=11.5761&temperature_unit=fahrenheit&current_weather=true`

This API call requests the hourly weather forecast for Munich with a geographical coordinate (latitude and longitude) set to its actual value, the temperature unit set to Fahrenheit (fahrenheit), and the current weather data included in the JSON output (current\_weather=true). By including only these necessary parameters, we can get a response that is as short as possible while still answering the user question. is not in the allowed domains: ['https://api.open-meteo.com/']

In [13]:
chain.invoke(
    "What is the weather last week  in Munich, Germany in degrees Celcius?"
)



> Entering new APIChain chain...
 https://api.open-meteo.com/v1/forecast?latitude=48.1351&longitude=11.5820&hourly=temperature_2m&daily=temperature_2m&timezone=Europe/Berlin&start_date=2022-06-30&end_date=2022-07-06&temperature_unit=celsius
{"reason":"Data corrupted at path ''. Cannot initialize ForecastVariableDaily from invalid String value temperature_2m.","error":true}

> Finished chain.


{'question': 'What is the weather last week  in Munich, Germany in degrees Celcius?',
 'output': ' The API call was not successful due to a data corruption issue with the temperature_2m variable. The requested information about the weather in Munich, Germany last week in degrees Celsius could not be retrieved.'}